# Case 4: Hybrid Architecture (Few Large + Many Small Effects)

This notebook follows the GWAS→PRS workflow but simulates a trait with a mixture of genetic effects: a few SNPs with strong effects plus many with tiny effects. This scenario reflects many real-world complex traits which have both "major genes" and polygenic background. We expect to see both clear genome-wide significant signals and additional polygenic signal that can be captured through broader PRS approaches.

### Step 0: Imports

Import required libraries for simulation, computation and plotting.


In [ ]:
# Step 0: Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from IPython.display import display

# Style for plots
plt.style.use('seaborn-v0_8-whitegrid')


### Step 1: Simulate Genotypes & Phenotypes
This case simulates a hybrid architecture: a few SNPs with large effects on the trait plus many SNPs with small effects. We split individuals 70/30 into discovery (training) and target (testing) sets. Noise is modest so genetics explains a large share of variance.

Configuration (dataclass):
- num_snps: total SNPs (default 1000)
- num_individuals: total people (split 70/30 into discovery/target)
- num_large_effect: number of large-effect SNPs (e.g., 5)
- num_small_effect: number of small-effect SNPs (e.g., 50)
- large_effect_size: per-allele effect for large-effect SNPs (e.g., 0.5)
- small_effect_size: per-allele effect for small-effect SNPs (e.g., 0.1)
- n_permutations: number of shuffles to build the empirical max |r| threshold (used in Step 4)
- broad_PGS_k: how many top-|r| SNPs to aggregate into the broad PRS (used in Step 5)
- noise_std: SD of environmental noise (set smaller here to yield high heritability)

What are we making?
- Genotypes: 0 / 1 / 2 copies of an allele at 1,000 SNPs
- Phenotype: Genetic component from 5 large-effect SNPs and 50 small-effect SNPs + modest noise

Why two groups?
- Discovery (train): estimate SNP–trait associations used to create the PRS
- Target (test): honest evaluation of the PRS

How do we simulate?
1) Draw allele frequency per SNP (uniform 0.05–0.5)
2) Generate genotypes (0/1/2) via binomial draws
3) Assign the first 5 SNPs large effects and the next 50 small effects; others are null
4) Compute g = G·β and add Gaussian noise (noise_std=0.5)
5) Use discovery to compute SNP–trait correlations; target is held out for PRS evaluation

**Beginner prompts (try these for deeper understanding):**
- “How do allele frequencies determine the probabilities of 0/1/2 genotypes?”  
- “What happens if I change the number of causal SNPs (e.g., 50 vs 300 vs 600)?”  
- "How does having both large and small effects impact our ability to detect signals?"
- "Why might a broad PRS outperform a strict PRS in some hybrid traits?"
- "What real-world traits might follow this hybrid pattern?"

In [ ]:
# Step 1: Config + Polygenic Simulation + Train Preview

from dataclasses import dataclass
import numpy as np
import pandas as pd
from IPython.display import display

@dataclass
class SimulationConfig:
    num_snps: int = 1000
    num_individuals: int = 1000  # Increase from 600 for better power
    num_large_effect: int = 5    # New parameter
    num_small_effect: int = 50   # New parameter
    large_effect_size: float = 0.5  # New parameter
    small_effect_size: float = 0.1  # New parameter
    noise_std: float = 0.5       # Reduced noise (from 1.0)
    n_permutations: int = 100
    broad_PGS_k: int = 100       # Increased from 50

config = SimulationConfig()
rng = np.random.default_rng(45)  # Different seed


# 1. Draw allele frequencies and genotypes (0/1/2)
allele_freqs = rng.uniform(0.05, 0.5, size=config.num_snps)
geno = np.empty((config.num_individuals, config.num_snps), dtype=np.int8)
for j, p in enumerate(allele_freqs):
    geno[:, j] = rng.binomial(2, p, size=config.num_individuals)

# 2. Split into discovery (train) and target (test) sets
n_train = int(0.7 * config.num_individuals)
train_idx = np.arange(n_train)
test_idx = np.arange(n_train, config.num_individuals)
geno_train, geno_test = geno[train_idx], geno[test_idx]

# 3. Define causal SNPs: large-effect + small-effect
large_effect_idx = np.arange(config.num_large_effect)  # First 5 SNPs
small_effect_idx = np.arange(config.num_large_effect, 
                          config.num_large_effect + config.num_small_effect)  # Next 50 SNPs
all_causal_idx = np.concatenate([large_effect_idx, small_effect_idx])

# Setup effect sizes
beta = np.zeros(config.num_snps)
beta[large_effect_idx] = config.large_effect_size
beta[small_effect_idx] = config.small_effect_size
# 4. Genetic component + noise with reduced variance
g_train = geno_train @ beta
g_test = geno_test @ beta

phen_train = g_train + rng.normal(0.0, config.noise_std, size=n_train)
phen_test = g_test + rng.normal(0.0, config.noise_std, size=config.num_individuals - n_train)

# Package for later steps
data = {
    "allele_freqs": allele_freqs,
    "geno_train": geno_train,
    "geno_test": geno_test,
    "phen_train": phen_train,
    "phen_test": phen_test,
    "large_effect_snps": large_effect_idx,
    "small_effect_snps": small_effect_idx,
    "all_causal_snps": all_causal_idx,
}

# Diagnostics (empirical h2 from train split)
var_g = g_train.var()
var_y = phen_train.var()
emp_h2 = var_g / var_y if var_y > 0 else np.nan
print(f"Train phenotype mean/var: {phen_train.mean():.3f} / {phen_train.var():.3f}")
print(f"Empirical h^2 ≈ {emp_h2:.3f}")
print(f"Large-effect SNPs: {len(large_effect_idx)} | Small-effect SNPs: {len(small_effect_idx)}")

# Preview first 5 rows
preview_snps = 8
df_train = pd.DataFrame(geno_train[:5, :preview_snps],
                        columns=[f"SNP_{i:04d}" for i in range(preview_snps)])
df_train["phen_train"] = phen_train[:5]
print("Train (wide) first rows:")
display(df_train)


### Interpreting the Training Set Preview

You’re seeing only the discovery (training) table. The target (test) data exists but isn’t shown here.

- Columns
  - SNP_0000, SNP_0001, …: Genotypes coded 0/1/2.
  - phen_train: Simulated trait = genetic component g + noise, where g = Σ_j (β_j × genotype_ij) with 5 large-effect SNPs and 50 small-effect SNPs.

- effect_size (what it means)
  - Per-allele additive change in the phenotype (slope). Moving 0→1→2 copies adds ≈ effect_size each step.
  - Simulation vs real data: in this notebook we choose effect_size (β). In real studies, β is unknown and estimated (β̂) via GWAS (per‑SNP regression), with standard errors and p‑values.
  - Larger |effect_size| generally produces bigger |r| and higher PRS R², but detectability also depends on sample size, allele frequency, and noise. In this notebook we use r as a simple proxy weight for β̂.
  
- Key points
  - By construction, SNPs 0–4 have large effects; SNPs 5–54 have small effects; others are null.
  - Train and test are separate draws; don’t use test data until evaluation.
  - We expect a couple of strong SNP signals plus broader sub-threshold signal from the many small effects.

- Next
  - Standardize (z-score) to compare SNP–trait correlations on a common scale.

In [ ]:
# Visualize the ground truth effect sizes
plt.figure(figsize=(8, 3))
x = np.arange(config.num_large_effect + config.num_small_effect)
y = np.concatenate([
    np.ones(config.num_large_effect) * config.large_effect_size,
    np.ones(config.num_small_effect) * config.small_effect_size
])
markerline, stemlines, baseline = plt.stem(x, y)  # removed use_line_collection
plt.setp(markerline, markersize=4, color='tab:blue')
plt.setp(stemlines, linewidth=1, color='tab:blue')
plt.axhline(0, color='gray', linestyle='-', alpha=0.3)
plt.ylabel('Effect Size (per allele)')
plt.xlabel('Causal SNP Index (0..{} large, then small)'.format(config.num_small_effect))
plt.title('True Effect Sizes (Hybrid Architecture)')
plt.tight_layout()
plt.show()

### Step 2: Standardization

We now convert raw genotype and phenotype values into **z-scores** in the discovery set.  A z-score answers the question: *"How many standard deviations above (+) or below (-) the mean is this value?"*

**Formula:**  
$z = (value - mean) / \mathrm{SD}$

**Why we do this before computing correlations:**
- Puts every SNP column and the phenotype on the **same scale** (mean 0, SD 1).
- Makes Pearson’s $r$ simply the **average product** of two standardized variables, which acts like an effect size.
- Allows us to compare SNP signals fairly — rare and common variants are no longer unfairly scaled.
- Keeps the *ordering* of individuals unchanged; we merely shift and rescale.

In a polygenic context with thousands of small effects, standardization is even more important: it ensures that none of the weak signals is artificially inflated or deflated simply due to allele frequency or measurement scale differences.


In [ ]:
# Step 2: Standardize Variables (Genotypes + Phenotypes)

def standardize_matrix(mat: np.ndarray):  # -> Tuple[np.ndarray, np.ndarray, np.ndarray]
    mean = mat.mean(axis=0)
    std = mat.std(axis=0)
    std[std == 0] = 1.0
    return (mat - mean) / std, mean, std

Z_geno_train, geno_mean_train, geno_std_train = standardize_matrix(data['geno_train'])
Z_geno_test, geno_mean_test_raw, geno_std_test_raw = standardize_matrix(data['geno_test'])  # independent standardization

phen_train = data['phen_train']
phen_test = data['phen_test']
Z_phen_train = (phen_train - phen_train.mean()) / phen_train.std()
Z_phen_test = (phen_test - phen_test.mean()) / phen_test.std()

# Diagnostics to show why raw vs standardized look similar
print("Raw phenotype   mean/std = {:.3f} / {:.3f}".format(phen_train.mean(), phen_train.std()))
print("Standardized    mean/std = {:.3f} / {:.3f}".format(Z_phen_train.mean(), Z_phen_train.std()))
print("Raw   min/max = {:.3f} / {:.3f}".format(phen_train.min(), phen_train.max()))
print("Z     min/max = {:.3f} / {:.3f}".format(Z_phen_train.min(), Z_phen_train.max()))

fig, axes = plt.subplots(1,2, figsize=(10,4))
# Raw distribution
axes[0].hist(phen_train, bins=30, color='skyblue', edgecolor='black')
axes[0].axvline(phen_train.mean(), color='k', linestyle='--', linewidth=1, label='Mean')
axes[0].set_title('Raw Phenotype (Discovery)')
axes[0].set_xlabel('Phenotype')
axes[0].legend()

# Standardized distribution (fixed axis to emphasize z-scale)
axes[1].hist(Z_phen_train, bins=30, color='salmon', edgecolor='black', density=True)
axes[1].axvline(0, color='k', linestyle='--', linewidth=1, label='Mean=0')
axes[1].set_xlim(-4,4)
axes[1].set_title('Standardized Phenotype (Discovery)')
axes[1].set_xlabel('Z-Phenotype')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Additional viz: one SNP before/after standardization (discovery set)
# Pick a SNP with allele frequency near 0.30 for variability
p = data['allele_freqs']
idx_snp = int(np.argmin(np.abs(p - 0.30)))
x_raw = data['geno_train'][:, idx_snp]
x_z = Z_geno_train[:, idx_snp]

fig, axes = plt.subplots(1, 2, figsize=(10, 3.5))

# Left: raw genotype distribution (0/1/2)
axes[0].hist(x_raw, bins=[-0.5, 0.5, 1.5, 2.5], color='steelblue', edgecolor='black')
axes[0].set_xticks([0,1,2])
axes[0].axvline(x_raw.mean(), color='k', linestyle='--', linewidth=1, label=f"Mean={x_raw.mean():.2f}")
axes[0].set_title(f"SNP_{idx_snp:04d} (raw)")
axes[0].set_xlabel('Genotype (0/1/2)')
axes[0].set_ylabel('Count')
axes[0].legend(frameon=False)

# Right: standardized genotype distribution (z-scores)
axes[1].hist(x_z, bins=30, color='indianred', edgecolor='black', density=True)
axes[1].axvline(0, color='k', linestyle='--', linewidth=1, label='Mean=0')
axes[1].axvline(1, color='gray', linestyle=':', linewidth=1, label='±1 SD')
axes[1].axvline(-1, color='gray', linestyle=':', linewidth=1)
axes[1].set_title(f"SNP_{idx_snp:04d} (standardized)")
axes[1].set_xlabel('Z-score')
axes[1].legend(frameon=False)

plt.tight_layout()
plt.show()
# ...existing code...

### Step 3: GWAS Correlations

**Goal.** Quantify the relationship between each SNP and the trait.

What to expect in Case 4 (hybrid):
- A few SNPs (the large-effect ones) with noticeably high |r| that should clear threshold.
- Many small-effect SNPs with modest |r| (often sub-threshold) that still contribute in aggregate via PRS.

**What we do.** For each SNP $j$, we compute the Pearson correlation $r_j$ between its standardized genotypes and the standardized phenotype in the discovery set.  Because both are z-scored, $r_j$ is just the average product of two series of standardized numbers.

**How to read $r$:**
- $r_j \approx 0$ means no detectable association.
- $r_j > 0$ means individuals with more minor alleles tend to have **higher** trait values.
- $r_j < 0$ means individuals with more minor alleles tend to have **lower** trait values.
- Larger $|r_j|$ implies a stronger SNP–trait link on a common scale (and $r_j^2$ is the in-sample variance explained by SNP $j$).

**Why so small?**  In a diffuse polygenic architecture, each true causal SNP has a tiny effect.  Their $|r_j|$ values are buried in sampling noise, so the largest observed $|r_j|$ will typically be modest (≈ 0.17) and may not exceed the permutation threshold.  The important information is in the **ranking** of $|r_j|$, not the individual magnitude.

**Intuition.**  To decide if a SNP and trait move together, imagine multiplying their standardized values person-by-person and averaging: if the values tend to be above average together, the average product is positive; if one tends to be high when the other is low, the average product is negative; if there’s no pattern, the average product is near zero.


In [ ]:
# Step 3: GWAS via per-SNP correlations

r_values = (Z_geno_train * Z_phen_train[:, None]).mean(axis=0)
r_abs = np.abs(r_values)
max_r = r_abs.max()
print(f"Maximum |r| = {max_r:.3f}")


In [ ]:
# Optional Step 3 viz: one SNP vs phenotype (discovery set)

import numpy as np
import matplotlib.pyplot as plt

idx = int(np.argmax(r_abs))  # SNP with largest |r|
snp_name = f"SNP_{idx:04d}"
is_large = idx in data['large_effect_snps']
is_small = idx in data['small_effect_snps']
effect_type = "large-effect" if is_large else "small-effect" if is_small else "null"

x_raw = data['geno_train'][:, idx]
y_raw = phen_train
x_z = Z_geno_train[:, idx]
y_z = Z_phen_train
r = float(r_values[idx]); r2 = r*r

rng = np.random.default_rng(123)
jit = (rng.random(x_raw.size) - 0.5) * 0.12  # jitter to separate 0/1/2 columns

fig, axes = plt.subplots(1, 2, figsize=(11, 4))

# Raw scale: shows per-allele shifts
axes[0].scatter(x_raw + jit, y_raw, s=8, alpha=0.5)
means = [y_raw[x_raw==g].mean() if np.any(x_raw==g) else np.nan for g in [0,1,2]]
axes[0].plot([0,1,2], means, 'r-o', lw=2, label='Group means')
axes[0].set_xticks([0,1,2]); axes[0].legend(frameon=False)
axes[0].set_xlabel('Genotype (0/1/2)'); axes[0].set_ylabel('Phenotype (raw)')
axes[0].set_title(f'{snp_name} vs phen_train ({effect_type})')

# Z scale: matches how r is computed
axes[1].scatter(x_z + jit, y_z, s=8, alpha=0.5)
# Group means in z space (plot at mean x_z for each genotype 0/1/2)
x_means = [x_z[x_raw==g].mean() if np.any(x_raw==g) else np.nan for g in [0,1,2]]
y_means = [y_z[x_raw==g].mean() if np.any(x_raw==g) else np.nan for g in [0,1,2]]
axes[1].plot(x_means, y_means, 'r-o', lw=2, label='Group means')
axes[1].legend(frameon=False)
axes[1].set_xlabel('Genotype (z-scored)'); axes[1].set_ylabel('Phenotype (z-scored)')
axes[1].set_title(f'{snp_name} (|r|={abs(r):.2f}, R²={r2:.2f})')

plt.tight_layout(); plt.show()

### How the scatterplot connects to Step 3 correlations - Hybrid Case

- Left panel (raw):
  - Three vertical bands (genotype 0/1/2). The red "group means" line shows the average phenotype for each genotype.
  - This hybrid architecture contains both large-effect and small-effect SNPs.
  - The SNP shown is likely a large-effect variant, showing a clear additive trend across genotype groups.
  - The slope (per-allele effect) is steep for large-effect SNPs but would be more modest for small-effect SNPs.

- Right panel (z-scored):
  - Both axes are standardized, which is exactly how we compute r in Step 3.
  - Pearson r for this SNP is the average product r = mean(x_z · y_z).
  - The higher |r| value (typically ~0.2-0.3 for large-effect SNPs) indicates a stronger genotype-phenotype association.
  - Large-effect SNPs explain more variance individually (higher R²), while small-effect SNPs contribute collectively.

- Why this matters:
  - In the Manhattan plot, we'll see clear peaks for large-effect SNPs rising above the threshold.
  - Small-effect SNPs create a slightly elevated "background" level compared to pure noise.
  - This hybrid architecture allows us to compare both significance-based (strict) and ranking-based (broad) PRS approaches.
  - The r values for large-effect SNPs provide strong weights for the PRS, while smaller effects contribute additively.

Note: The hybrid architecture combines features of both Case 1 (sparse) and Case 2 (polygenic), with a mix of clear strong signals and numerous weaker signals that together improve prediction.

### Mini-GWAS framing (what we’re mimicking)

- A GWAS tests each SNP across the genome for association with a phenotype (one SNP at a time).  
- Real GWAS uses regression (**β, SE, p-value**) and includes covariates (age, sex, ancestry PCs) to control confounding.  
- In this exercise, we use **z-scores** and compute a simple **per-SNP correlation (r)** as a stand-in for GWAS effect size.  
- Output of this step is a “summary stats–like” table: **SNP ID, r, |r|**.  
- Next, we’ll visualize all SNPs together (Manhattan plot) and set a significance cut line via **permutation**—analogous to GWAS genome-wide thresholds.  

**TL;DR:** Step 3 ≈ a **mini-GWAS** pass over SNPs to get per-SNP effects we can carry forward.  


### Step 4: Permutation (simple) + Manhattan Plot (Hybrid)

From one SNP to all SNPs
- Step 3 computed an r per SNP; the Manhattan plot shows |r| across all SNPs on a common z-scale.

What we do (simple permutation)
- We ask: “How tall could the biggest |r| be just by chance?”
- Shuffle the phenotype B times.
- Each time, compute |r| across SNPs and record the maximum.
- Use the 95th percentile of these maxima as the dashed threshold.

How to read the Manhattan (hybrid)
- Dots = SNPs; height = |r|. Dashed orange line = permutation threshold.
- Purple stars = Top‑K by |r| (broad PRS).
- Expect a few peaks above the line (large effects) plus a slightly elevated “floor” from many small effects.

PRS note
- Strict PRS: SNPs ≥ threshold. Broad PRS: Top‑K by |r|.

Beginner tip
- “Mountains rising from hills”: big peaks (large effects) on top of a low, rolling background (small effects).

In [ ]:
# Step 4: Compact permutation + Manhattan (single line + Top‑K overlay)
def perm_threshold(Z_geno: np.ndarray, y: np.ndarray, B: int = 100, q: float = 0.95, seed: int = 0) -> float:
    rng = np.random.default_rng(seed)
    max_abs = np.empty(B, dtype=float)
    for b in range(B):
        y_shuf = rng.permutation(y)
        r_shuf = (Z_geno * y_shuf[:, None]).mean(axis=0)
        max_abs[b] = np.abs(r_shuf).max()
    return float(np.quantile(max_abs, q))

ALPHA = 0.05
B = config.n_permutations if hasattr(config, "n_permutations") else 100
threshold = perm_threshold(Z_geno_train, Z_phen_train, B=B, q=1-ALPHA, seed=0)

# Selection summary for strict vs broad PRS
r_abs = np.abs(r_values)
topK = config.broad_PGS_k
topK_idx = np.argsort(r_abs)[-topK:]
selected_idx = np.flatnonzero(r_abs >= threshold)

print(f"Permutation threshold (95%): {threshold:.3f} | Above threshold: {selected_idx.size} | Top-K (broad PRS) = {topK}")

# Manhattan plot (hybrid)
plt.figure(figsize=(10, 5))
x = np.arange(config.num_snps)
plt.scatter(x, r_abs, s=10, c='black', alpha=0.6, label='All SNPs')
plt.scatter(topK_idx, r_abs[topK_idx], s=40, c='purple', alpha=0.9, marker='*', label=f'Top {topK} by |r| (broad PRS)')
plt.axhline(threshold, color='orange', linestyle='--', linewidth=1.5, label='95% permutation threshold')
plt.xlabel('SNP index'); plt.ylabel('Absolute correlation |r|')
plt.title('Manhattan Plot (Hybrid: threshold + Top‑K overlay)')
plt.legend(loc='upper right', frameon=False)
plt.tight_layout(); plt.show()

### From GWAS results to Manhattan (our version)

- Classic GWAS Manhattan plots **−log10(p)** by genomic position; taller peaks = stronger evidence.  
- Here, we plot **|r|** instead of −log10(p). The goal is the same: a genome-wide view of signal strength.  
- GWAS uses fixed significance lines (e.g., *5×10⁻⁸*). We use a **permutation-based line** that reflects our data.  
- Selection rule: take all SNPs above the line; if none cross, use a **top-K by |r| fallback** (keep the sign of r for direction).  
- Same idea, simpler ingredients: our permutation line plays the role of a **GWAS significance threshold**.  


### Step 5: Build and Evaluate a Polygenic Score

Goal: Compare a strict PRS using only genome-wide hits vs a broad PRS that also captures small effects.

What we do:
1. Strict PRS: include SNPs that pass the permutation threshold (should be ~5 large-effect SNPs)
2. Broad PRS: include top-K SNPs by |r| (e.g., K=100) to capture large + small effects
3. Evaluate performance: correlation (r, R²) and decile stratification
4. Visualize: Scatter and decile comparison

In [ ]:
# Step 5: Build and evaluate polygenic scores (strict and broad)

# Strict PRS: only SNPs passing threshold
sig_idx = np.where(r_abs >= threshold)[0]
strict_label = f'Strict ({len(sig_idx)} SNPs)'
strict_prs = Z_geno_test[:, sig_idx] @ r_values[sig_idx] if len(sig_idx) > 0 else np.zeros(len(Z_phen_test))
strict_prs = (strict_prs - strict_prs.mean()) / strict_prs.std() if strict_prs.std() > 0 else strict_prs

# Broad PRS: top-K SNPs regardless of threshold  
broad_idx = np.argsort(r_abs)[-config.broad_PGS_k:]
broad_label = f'Broad (top-{config.broad_PGS_k})'
broad_prs = Z_geno_test[:, broad_idx] @ r_values[broad_idx]
broad_prs = (broad_prs - broad_prs.mean()) / broad_prs.std()

# Evaluate both against phenotype
strict_R = float(np.corrcoef(strict_prs, Z_phen_test)[0,1]) if strict_prs.std() > 0 else 0.0
broad_R = float(np.corrcoef(broad_prs, Z_phen_test)[0,1])
strict_R2 = strict_R * strict_R
broad_R2 = broad_R * broad_R

print(f"Significant SNPs (threshold): {len(sig_idx)}")
print("PRS comparison:")
print(f"  - {strict_label}: r = {strict_R:.3f}, R² = {strict_R2:.3f}")
print(f"  - {broad_label}: r = {broad_R:.3f}, R² = {broad_R2:.3f}")

In [ ]:
# Visualization: Compare scatter plots
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.scatter(strict_prs, Z_phen_test, s=8, alpha=0.4, color='orangered')
plt.xlabel('Strict PRS (z-score)')
plt.ylabel('Phenotype (z-score)')
plt.title(f'Strict PRS (r={strict_R:.2f}, R²={strict_R2:.2f})')

plt.subplot(1, 3, 2)
plt.scatter(broad_prs, Z_phen_test, s=8, alpha=0.4, color='royalblue')
plt.xlabel('Broad PRS (z-score)')
plt.ylabel('Phenotype (z-score)')
plt.title(f'Broad PRS (r={broad_R:.2f}, R²={broad_R2:.2f})')

# Create side-by-side decile plots
plt.subplot(1, 3, 3)

# Create strict PRS deciles
edges_strict = np.quantile(strict_prs, np.linspace(0,1,11))
dec_strict = np.digitize(strict_prs, edges_strict[1:-1], right=True)
mean_strict = [Z_phen_test[dec_strict == d].mean() if np.sum(dec_strict==d) > 0 else np.nan for d in range(10)]
se_strict = [Z_phen_test[dec_strict == d].std(ddof=1)/np.sqrt(np.sum(dec_strict==d)) 
            if np.sum(dec_strict==d) > 1 else np.nan for d in range(10)]
gap_strict = mean_strict[-1] - mean_strict[0]

# Create broad PRS deciles
edges_broad = np.quantile(broad_prs, np.linspace(0,1,11))
dec_broad = np.digitize(broad_prs, edges_broad[1:-1], right=True)
mean_broad = [Z_phen_test[dec_broad == d].mean() if np.sum(dec_broad==d) > 0 else np.nan for d in range(10)]
se_broad = [Z_phen_test[dec_broad == d].std(ddof=1)/np.sqrt(np.sum(dec_broad==d)) 
           if np.sum(dec_broad==d) > 1 else np.nan for d in range(10)]
gap_broad = mean_broad[-1] - mean_broad[0]

# Plot both decile trends
plt.errorbar(range(1,11), mean_strict, yerr=se_strict, fmt='-o', capsize=3, 
            color='orangered', label=f'Strict PRS (Δ₁₀₋₁={gap_strict:.2f})')
plt.errorbar(range(1,11), mean_broad, yerr=se_broad, fmt='-o', capsize=3, 
            color='royalblue', label=f'Broad PRS (Δ₁₀₋₁={gap_broad:.2f})')
plt.title(f'PRS Decile Comparison')
plt.xlabel('PRS Decile (1=lowest, 10=highest)')
plt.ylabel('Mean Phenotype (z-score)')
plt.legend()

plt.tight_layout()
plt.show()

### Step 5 Interpretation

**Performance metrics:**
- Strict PRS: Using only significant SNPs (approximately 5)
  - Correlation: r ≈ 0.7, R² ≈ 0.49
  
- Broad PRS: Using top 100 SNPs by |r|
  - Correlation: r ≈ 0.75, R² ≈ 0.56

**What this means:**
- **Clear strong signals**: The strict PRS captures the large-effect SNPs that stand out in the Manhattan plot
- **Additional polygenic signal**: The broad PRS improves performance by including small-effect variants
- **Two-tier genetic architecture**: This hybrid scenario demonstrates that both approaches have value

**Understanding the decile comparison:**
- Both PRS models show strong stratification across deciles
- The strict PRS has a steep gradient driven by the few large effects
- The broad PRS shows a slightly improved gradient by capturing both large and small effects
- The gap between lowest and highest deciles (Δ₁₀-₁) is substantially larger than in purely polygenic traits

**Why this matters:**
This hybrid architecture represents many real-world complex traits where a few loci have outsized influence while numerous small effects contribute to the polygenic background. Understanding both components can inform targeted interventions (for major genes) and population-level approaches (for polygenic background).

### Conclusion: What We Learned from Case 4

**Journey summary**  
We explored a hybrid genetic architecture combining few large-effect variants with many small-effect variants. This architecture mirrors many real-world complex traits that have both "major genes" and polygenic background.

**What we accomplished**
1. Simulated a trait with dual genetic architecture: 5 strong-effect SNPs plus 50 small-effect SNPs
2. Standardized variables to enable fair comparisons across SNPs of varying frequencies
3. Observed a Manhattan plot with clear peaks (large effects) and a "polygenic floor" (small effects)
4. Evaluated two PRS approaches: strict (significant SNPs only) and broad (top 100 SNPs)
5. Demonstrated that while the strict PRS performs well, the broad approach captures additional variance
6. Visualized how both approaches effectively stratify individuals across risk deciles

**Key insights**
- Hybrid architectures benefit from two-tier analytic approaches
- The strict PRS offers interpretability by focusing on validated signals
- The broad PRS maximizes predictive power by capturing both large and small effects
- The optimal approach depends on the specific goals (explanation vs prediction)
- Decile stratification is particularly strong in hybrid traits, offering clear risk differentiation

**Why this matters**  
Many clinically relevant traits show this hybrid architecture - examples include conditions like type 2 diabetes, breast cancer, and heart disease, where known major risk variants exist alongside polygenic background. Understanding both components can inform both targeted interventions and population-level risk assessment.

**Taking it further**
- Explore differential weighting schemes that prioritize validated variants
- Investigate prediction performance across different population backgrounds
- Consider gene-gene interaction effects between major variants and polygenic background
- Evaluate prediction accuracy across different segments of the risk distribution

**Comparison to Other Cases**
- Unlike Case 1 (sparse), signal comes from both major and minor genetic factors
- Unlike Case 2 (polygenic), some SNPs are clearly detectable on their own
- This hybrid architecture combines the benefits of both: clear targets for follow-up (large effects) and improved prediction through aggregation (small effects)

# Complete the Reflection & Comparison Questions in Shared Slides in Groups in Canvas